In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [40]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [41]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [42]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    
    items = np.array(items_weights['item_id'])
    t = items_weights['weight'].sum()
    p = np.array(items_weights['weight'])/t
    
    recs = np.random.choice(items, size=n, replace=False, p=p)
    
    return recs.tolist()



In [43]:
%%time

item_weighted = data_train.groupby('item_id')['sales_value'].count().reset_index()
item_weighted.columns=['item_id', 'weight']
item_weighted['weight'] = np.log(item_weighted['weight'])


res = weighted_random_recommendation(item_weighted, n=5)

print(res)

[1017423, 1057949, 8205620, 843574, 12352248]
CPU times: user 91.3 ms, sys: 21.6 ms, total: 113 ms
Wall time: 117 ms


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [44]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [45]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [46]:
%%time
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5000))

result.head(2)

CPU times: user 3.7 s, sys: 176 ms, total: 3.87 s
Wall time: 3.9 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940631, 936117, 267854, 1052094, 1147335, 992..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9419164, 924518, 12949308, 17239657, 12262570..."


In [47]:
item_weighted = data_train.groupby('item_id')['sales_value'].count().reset_index()
item_weighted.columns=['item_id', 'weight']
item_weighted['weight'] = np.log(item_weighted['weight'])


result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weighted, n=5000))

result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940631, 936117, 267854, 1052094, 1147335, 992...","[991268, 983584, 850656, 1088345, 9487323, 204..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9419164, 924518, 12949308, 17239657, 12262570...","[9651061, 9296924, 1010164, 940023, 830526, 83..."


In [48]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5000)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 108 ms, sys: 19.5 ms, total: 127 ms
Wall time: 127 ms


,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940631, 936117, 267854, 1052094, 1147335, 992...","[991268, 983584, 850656, 1088345, 9487323, 204...","[6534178, 6533889, 1029743, 6534166, 1082185, ..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9419164, 924518, 12949308, 17239657, 12262570...","[9651061, 9296924, 1010164, 940023, 830526, 83...","[6534178, 6533889, 1029743, 6534166, 1082185, ..."


In [49]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [50]:

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [51]:
%%time

model = ItemItemRecommender(K=50, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 32.6 s, sys: 808 ms, total: 33.4 s
Wall time: 40.1 s


In [52]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

CPU times: user 745 ms, sys: 10.8 ms, total: 755 ms
Wall time: 857 ms


In [56]:
result.head(5)

def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    return precision(recommended_list, bought_list)

In [57]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0006:random_recommendation
0.002:weighted_random_recommendation
0.1552:popular_recommendation
0.1664:itemitem
